# Prior functionality

In this notebook I will demonstrate a basic redshift prior and show how more complex priors might be developed. Essentially the user needs to define a set of weights for each model and pass these weights to the prior for a given source.

In [ ]:
import lephare as lp
import numpy as np
from matplotlib import pylab as plt

%matplotlib inline

In [ ]:
config = lp.default_cosmos_config.copy()
config["Z_STEP"] = "0.1,0.,5."  # To make notebook very fast to run
keymap = lp.all_types_to_keymap(config)

In [ ]:
lp.prepare(keymap)

In [ ]:
photz = lp.PhotoZ(keymap)

In [ ]:
len(photz.gridz)

In [ ]:
len(photz.fullLib)

In [ ]:
def fulllib_to_weights(fullLib):
    """Function which takes the full library of models as an input and returns a prior weight for each.

    This is a means to apply a fully generalisable prior which can be called independently for each source.

    In this case as a basic example I impose a maximum redshift of 2.

    Parameters
    ==========

    fullLib : list of lephare.QSOSED, lephare.StarSED, lephare.GalSED
        The full library of models to fit

    Returns
    =======

    weights : list of floats
        the list of prior weights to apply to each model.
    """
    weights = np.array([1 for l in fullLib])
    # Get the redshift using the index
    # Seems odd. Is this really the best way to get the redshift?
    red = np.array([photz.fullLib[n].red for n in np.arange(len(photz.fullLib))])

    weights[red > 2] = 1.0e-9

    return weights


fulllib_to_weights(photz.fullLib)[:5]

In [ ]:
# Lets look at what we have access to determine the prior weight

In [ ]:
qso_ex = photz.fullLib[0]
star_ex = photz.fullLib[900]
gal_ex = photz.fullLib[3000]

In [ ]:
# The redshift
gal_ex.red

In [ ]:
red = [photz.fullLib[n].red for n in np.arange(len(photz.fullLib))]

In [ ]:
red[:5]

In [ ]:
np.unique(red)

In [ ]:
# The model numbers to set priors on a given template
nummod = [l.nummod for l in photz.fullLib]
np.unique(nummod)

In [ ]:
# The ebv are all zero. I think this is because they are applied to the photometry not the model
photz.fullLib[3000].ebv

In [ ]:
photz.fullLib[:5]

In [ ]:
# Set the prior function via pybind

## Compare results with and without the simple redshift prior

Here we will run twice with the redshift prior above applied or not to show how it impacts the predicted redshifts.

In [ ]:
cat = np.loadtxt(f"{lp.LEPHAREDIR}/examples/COSMOS.in")
id = cat[:, 0]
fluxes = cat[:, 1:60:2]
efluxes = cat[:, 2:61:2]
context = cat[:, 61]
zspec = cat[:, 62]
print("Check format with context and zspec :", context, zspec)

In [ ]:
w = fulllib_to_weights(photz.fullLib)

In [ ]:
srclist = []
zspec_mask = np.logical_and(zspec > 0.01, zspec < 6)
for i in np.where(zspec_mask)[0]:
    oneObj = lp.onesource(i, photz.gridz)
    oneObj.readsource(str(id[i]), fluxes[i, :], efluxes[i, :], int(context[i]), zspec[i], " ")
    oneObj.priorObj.apply_weights = 1
    oneObj.priorObj.weights = w
    photz.prep_data(oneObj)
    srclist.append(oneObj)
print("Sources with a spec-z: ", len(srclist))

In [ ]:
# Run the photoz

In [ ]:
a0 = photz.run_autoadapt(srclist)
offsets = ",".join(np.array(a0).astype(str))
offsets = "# Offsets from auto-adapt: " + offsets + "\n"
print(offsets)

In [ ]:
photozlist = []
for i in range(100):
    oneObj = lp.onesource(i, photz.gridz)
    oneObj.readsource(str(id[i]), fluxes[i, :], efluxes[i, :], int(context[i]), zspec[i], " ")
    oneObj.priorObj.apply_weights = 1
    oneObj.priorObj.weights = w
    photz.prep_data(oneObj)
    photozlist.append(oneObj)
print("Number of sources to be analysed: ", len(srclist))

In [ ]:
photz.run_photoz(photozlist[:100], a0)

In [ ]:
t = photz.build_output_tables(photozlist[:100], para_out=None, filename="testPrior.fits")

In [ ]:
t[:5]

In [ ]:
plt.hist(t["Z_BEST"], bins=50)
plt.xlabel("z with z<2 prior")

In [ ]:
photozlist = []
for i in range(100):
    oneObj = lp.onesource(i, photz.gridz)
    oneObj.readsource(str(id[i]), fluxes[i, :], efluxes[i, :], int(context[i]), zspec[i], " ")
    # oneObj.priorObj.apply_weights=1
    # oneObj.priorObj.weights=w
    photz.prep_data(oneObj)
    photozlist.append(oneObj)
print("Number of sources to be analysed: ", len(srclist))

photz.run_photoz(photozlist[:100], a0)

t2 = photz.build_output_tables(photozlist[:100], para_out=None, filename="testPrior.fits")
plt.hist(t2["Z_BEST"], bins=50)
plt.xlabel("z without z<2 prior")